In [2]:
from ib_insync import *
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd

In [8]:
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

<IB connected to 127.0.0.1:7497 clientId=1>

In [ ]:
def loadData(contract, name, start, end, barSize='1 min'):
    bars = ib.reqHistoricalData(
        contract, endDateTime=end.strftime('%Y%m%d %H:%m:%S'), 
        durationStr='1 D', barSizeSetting=barSize, whatToShow='MIDPOINT', useRTH=True)
    end -= timedelta(days=1)
    df = util.df(bars)
    k = 0
    while end > start:
        print('Got ' +end.strftime('%Y-%m-%d'))
        bars = ib.reqHistoricalData(
            contract, endDateTime=end.strftime('%Y%m%d %H:%m:%S'), 
            durationStr='1 D', barSizeSetting=barSize, whatToShow='MIDPOINT', useRTH=True)
        end -= timedelta(days=1)
        df = df.append(util.df(bars))
        k += 1
        # checkpoints
#         if k % 100 == 0:
#             df.drop(['volume', 'average', 'barCount'], axis=1).sort_values('date', axis=0).to_csv(f'{name}_{k/100}.csv', index=False)
    return df.drop(['volume', 'average', 'barCount'], axis=1)\
            .sort_values('date', axis=0)\
            .drop_duplicates(subset=['date'], inplace=True)

In [4]:
futures = ['ES','NQ','CL','NG','HO','XB','HG','GC','SI','ZN','ZF','ED']

In [5]:
for ticker in futures:
    data = loadData(contract, ticker, datetime(2018, 11, 1), datetime(2021, 11, 15))
    df.rename(columns={'date' : 'time'})
    data.to_csv(f'{ticker}.csv', index=False)

Error 321, reqId 3: Error validating request.-'bT' : cause - Please enter exchange, contract: Future(symbol='ES', lastTradeDateOrContractMonth='CME')
Error 321, reqId 4: Error validating request.-'bT' : cause - Please enter exchange, contract: Future(symbol='ES', lastTradeDateOrContractMonth='CME')


Got 2021-11-14


AttributeError: 'NoneType' object has no attribute 'append'

In [4]:
def loadFutures(symbol, exchange, last_date, longevity, release_interval, start, end, barSize='1 min'):
    future_end = last_date
    while future_end > start:
        future_start = future_end - longevity
        end_month = future_end.strftime('%Y%m')
        contract = Future(symbol, future_end.strftime('%Y%m'), exchange)
        current_day = min(end, last_date)
        bars = ib.reqHistoricalData(
            contract, endDateTime=end.strftime('%Y%m%d %H:%m:%S'), 
            durationStr='1 D', barSizeSetting=barSize, whatToShow='TRADES', useRTH=True)
        df = util.df(bars)
        while current_day > min(start, future_start):
            current_day -= timedelta(days=1)
            bars = ib.reqHistoricalData(
                contract, endDateTime=end.strftime('%Y%m%d %H:%m:%S'), 
                durationStr='1 D', barSizeSetting=barSize, whatToShow='TRADES', useRTH=True)
            try:
                data = util.df(bars)
                df = df.append(util.df(bars))
                print('Got ' +current_day.strftime('%Y-%m-%d'))
            except:
                print('Did NOT get ' +current_day.strftime('%Y-%m-%d'))
                pass
        df.rename(columns={'date' : 'time'}, inplace=True)
        df = df.sort_values('time', axis=0).drop_duplicates(subset=['time'])
        df.to_csv(f"{symbol}-{future_end.strftime('%Y')[2:]}.{future_end.strftime('%m')}.csv", index=False)
        future_end -= release_interval

In [21]:
def loadFullFutures(symbol, exchange, last_date, longevity, release_interval, releases, barSize='1 min'):
    future_end = last_date
    while releases > 0:
        future_start = future_end - longevity
        end_month = future_end.strftime('%Y%m')
        contract = Future(symbol, future_end.strftime('%Y%m'), exchange)
        current_day = last_date
        bars = ib.reqHistoricalData(
            contract, endDateTime=current_day.strftime('%Y%m%d %H:%m:%S'), 
            durationStr='1 D', barSizeSetting=barSize, whatToShow='TRADES', useRTH=True)
        df = util.df(bars)
        while current_day > future_start:
            current_day -= timedelta(days=1)
            bars = ib.reqHistoricalData(
                contract, endDateTime=current_day.strftime('%Y%m%d %H:%m:%S'), 
                durationStr='1 D', barSizeSetting=barSize, whatToShow='TRADES', useRTH=True)
            try:
                data = util.df(bars)
                df = df.append(util.df(bars))
                print('Got ' +current_day.strftime('%Y-%m-%d'))
            except:
                print('Did NOT get ' +current_day.strftime('%Y-%m-%d'))
                pass
        df.rename(columns={'date' : 'time'}, inplace=True)
        df = df.sort_values('time', axis=0).drop_duplicates(subset=['time'])
        df.to_csv(f"{symbol}-{future_end.strftime('%Y')[2:]}.{future_end.strftime('%m')}.csv", index=False)
        future_end -= release_interval
        releases -= 1

In [22]:
bars = loadFullFutures('NQ', 'GLOBEX', datetime(2021, 12, 17), relativedelta(months=15), relativedelta(months=3), 3)

ConnectionError: Not connected

In [82]:
df = pd.read_csv('ES-21.12.csv', index_col='time')
# df = df.drop(columns='Unnamed: 0')
# df = df.rename(columns={'date' : 'time'})
# df = df.sort_values('time', axis=0)
# df = df.drop_duplicates(subset=['time'])
# df.to_csv('ES-21.12.csv', index=False)

In [83]:
df.head()

,open,high,low,close,volume,average,barCount
time,,,,,,,
2021-11-19 17:30:00,4700.75,4702.25,4695.00,4696.75,21634.0,4699.400,4633
2021-11-19 17:31:00,4696.75,4697.50,4693.25,4693.25,11554.0,4695.775,2984
2021-11-19 17:32:00,4693.50,4696.50,4693.25,4696.25,7553.0,4695.250,1920
2021-11-19 17:33:00,4696.00,4696.75,4692.25,4692.75,5412.0,4694.625,1401
2021-11-19 17:34:00,4693.00,4695.00,4691.50,4695.00,5903.0,4693.250,1537


In [17]:
bars = loadFullFutures('NQ', 'GLOBEX', datetime(2021, 9, 17), relativedelta(months=15), relativedelta(months=3), 1)

NameError: name 'end' is not defined

In [88]:
contract = Future('ES', '202009', 'CME')
bars = ib.reqHistoricalData(
            contract, endDateTime='20200517 00:00:00', 
            durationStr='1 D', barSizeSetting='1 min', whatToShow='TRADES', useRTH=True)

Error 200, reqId 1019: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202009', exchange='CME')
Peer closed connection.
